In [ ]:
%cd ~
import matplotlib
matplotlib.use("PDF")
import pyfits,glob,time,scipy, sys
import scipy.interpolate
from scipy.special import erf
from pylab import *
from subprocess import call
from IPython import parallel
from scipy.optimize import fmin_powell, minimize
from scipy.special import airy,jv
from scipy.integrate import quad

topdir='/Users/cslage/Research/LSST/code/GUI/'
#thedir=topdir+'brighter_fatter/20170831_002_spots/'
thedir=topdir+'brighter_fatter/20171128_002_spots_VBB60/'
sys.path.append('/Users/cslage/Learning/code/languages/python/extensions/forward_model_convolved')
import  forward_convolved

%cd $thedir

configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'


In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()
    
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

In [ ]:
# This definition runs sextractor in parallel and can be given to IPython's parallel map function along with
#   the file list
def ovsub_runsex_makereg(fitsfilename):
    import pyfits
    import numpy as np
    from subprocess import call
    topdir='/Users/cslage/Research/LSST/code/GUI/'
    thedir=topdir+'profiles'
    %cd $thedir

    configfile=topdir+'sextractor/default-array_dither.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    maskdir=topdir+'sextractor/masks/'

    def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
        overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
        image=image[:y_overscan_start,:x_overscan_start]
        finalimg=image-np.matrix(np.median(overscan,axis=1)).T*np.ones(np.shape(image)[1])
        return finalimg

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
    for i in range(1,17):
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname)
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022)   # cut off the overscan
        outname=fitsfilename[:-5]+extname+'.fits'
        pyfits.writeto(outname,overscansubimg,clobber=True)
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat'])
        make_reg_from_ldac(outname+'.cat','NUMBER')

In [ ]:
zfilelist=sort(glob.glob(thedir+'ITL-3800C-002_spot_spot_250_20171128??????.fits'))
print len(zfilelist)

In [ ]:
for fitsfilename in zfilelist: 
    tfile1=time.time() 
    for i in [13]:
        tstart=time.time() 
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME'] 
        img=pyfits.getdata(fitsfilename,extname) 
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
        # cut off the overscan 
        outname=fitsfilename[:-5]+extname+'.fits' 
        pyfits.writeto(outname,overscansubimg,clobber=True) 
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat']) 
        make_reg_from_ldac(outname+'.cat','NUMBER') 
        tend=time.time() 
        print extname+" time: "+str(tend-tstart)[:4] 
        print "Time taken for file "+str(fitsfilename[-26:-23])+": "+str(time.time()-tfile1)

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 50
    xcoomax = 450
    ycoomin = 1400
    ycoomax = 1900
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pyfits.getheader(fitsfilename,segmentnumber)
    extname = hdr['EXTNAME']
    img=pyfits.getdata(fitsfilename,extname) 
    overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
    catname=fitsfilename[:-5]+extname+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = overscansubimg[ymin:ymax, xmin:xmax]
            background = median(stamp)
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i]) - background                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xoff#xcoord - xint - 1.0
            spotlist.yoffset[n] = yoff#ycoord - yint - 1.0     
                    
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom


In [ ]:
from scipy.ndimage import shift
def DrizzleSpotList(spotlist, nzoom):
    stampstack=zeros((spotlist.nx*nzoom,spotlist.ny*nzoom))
    regridstamp=zeros((spotlist.nx*nzoom,spotlist.ny*nzoom))
    for k in range(spotlist.nstamps):
        xshift,yshift=round(spotlist.xoffset[k]*nzoom),round(spotlist.yoffset[k]*nzoom)
        for i in range(spotlist.nx):
            for j in range(spotlist.ny):
                i0,i1=nzoom*i,nzoom*(i+1)
                j0,j1=nzoom*j,nzoom*(j+1)
                regridstamp[i0:i1,j0:j1]=spotlist.data[i,j,k]
        shiftstamp=shift(regridstamp,(-xshift,-yshift),order=0)
        stampstack+=shiftstamp
    stampstack=stampstack/spotlist.nstamps
    return stampstack

def moments(data, center):
    y, x = np.indices((data.shape))
    x = (x - center[0] + 0.5) / nzoom
    y = (y - center[1] + 0.5) / nzoom
    mx = (data * x).sum() / data.sum()
    my = (data * y).sum() / data.sum()
    mx2 = (data * (x-mx) * (x-mx)).sum() / data.sum()
    my2 = (data * (y-my) * (y-my)).sum() / data.sum()
    mxy = (data * (x-mx) * (y-my)).sum() / data.sum()
    return [mx, my, mx2, my2, mxy]

def radial_profile(data, center):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    rx=linspace(0,halfstampsize*np.sqrt(2),round(halfstampsize*nzoom*np.sqrt(2)))
    return rx,radialprofile 


In [ ]:
nx = 11
ny = 11
numspots =400
segmentnumber = 13
fitsfilename = zfilelist[0]
print fitsfilename
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.7, 1.4)

nzoom = 10
halfstampsize = (nx + 1) / 2
stampstack = DrizzleSpotList(spotlist, nzoom)
[mx, my, mx2, my2, mxy] = moments(stampstack, (55,55))
print mx, my, mx2, my2, mxy

figure(figsize=(12,6))
subplot(121)
imshow(stampstack,cmap='rainbow',aspect='equal',origin='lower'),colorbar()

for i in range(1,2*halfstampsize):
    axvline(i*nzoom,color='w')
    axhline(i*nzoom,color='w')
axis('off')
subplot(122)
rx,rp=radial_profile(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))

plot(rx,rp/rp.max())
#yscale('log')

foox=np.linspace(0,8,100)
foosig=1.25
fooy=np.exp(-foox**2/(2.0*foosig**2))
plot(foox,fooy)

show()
close()



In [ ]:
def CGIntegrand(y, xl, yl, c):
    # Convolved Gaussian Integrand.
    s2 = sqrt(2.0)
    yc = sqrt(c*c - y*y)
    yp = yc + xl
    ym = yc - xl
    f = yp * erf(yp / s2) - ym * erf(ym / s2)
    f += s2 * (exp(-yp * yp / 2.0) - exp(-ym * ym / 2.0)) / sqrt(pi)
    f *= erf((y+yl) / s2) - 1.0
    f -= 2.0 * yc * erf((y+yl) / s2)
    f += pi * c / 2.0
    return f / (4.0 * pi * c * c)


def Area(xl, xh, yl, yh, sigmax, sigmay, c):
    # Calculates how much of a 2D Convolved Gaussian falls within a rectangular box
    f = quad(CGIntegrand, -c, c, args=(xl/sigmax,yl/sigmay,c))[0]
    f += quad(CGIntegrand, -c, c, args=(xh/sigmax,yh/sigmay,c))[0]
    f -= quad(CGIntegrand, -c, c, args=(xl/sigmax,yh/sigmay,c))[0]
    f -= quad(CGIntegrand, -c, c, args=(xh/sigmax,yl/sigmay,c))[0]
    return f

def CGFOM(params):
    [sigma, ctest] = params
    if sigma < 0.0 or ctest < 0.0: return 1.0E12
    NormFactor = CGFunc(0.0,0.0,sigma,sigma,c)
    fom = 0.0
    for i,x in enumerate(rx):
        cg = CGFunc(x,0,sigma, sigma, ctest) / NormFactor
        fom += square(cg-rp[i]/rp.max())
    #print sigma, ctest, fom
    return fom

def CGSpotIntegrand(y1, x, y, sigmax, sigmay, c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    f = sqrt(pi/2.0) * sigmax *(erf((sqrt(c*c-y1*y1)+x)/ssx) - erf((x-sqrt(c*c-y1*y1))/ssx))
    f *= exp(-(y-y1)*(y-y1) / (ssy*ssy))
    return f

def CGFunc(x, y, sigmax, sigmay, c):
    return quad(CGSpotIntegrand, -c, c, args=(x,y,sigmax,sigmay,c))[0]

def ConvolvedGaussian(rx,sigmax, sigmay,c):
    NormFactor = CGFunc(0.0,0.0,sigma,sigma,c)
    fooy = []
    for r in rx:
        fooy.append(CGFunc(r,0.0,sigmax,sigmay,c) / NormFactor)
    return(array(fooy))

def Gaussian(x, y, sigmax=1.0, sigmay=1.0):
    ssigx = sigmax * sqrt(2.0)
    ssigy = sigmay * sqrt(2.0)
    sr = sqrt((x/ssigx)**2 + (y/ssigy)**2)
    return exp(-sr**2)

def GFOM(params):
    fom = 0.0
    [sigma] = params
   
    for i,x in enumerate(rx):
        cg = Gaussian(x,0,sigmax=sigma, sigmay=sigma)
        #print x, cg, rp[i]/rp.max()
        fom += square(cg-rp[i]/rp.max())
    #print sigma, ctest, fom
    return fom


In [ ]:
figure(figsize=(12,6))
subplots_adjust(hspace=0.5)
subplot(2,1,1)
title("Spot Profile")
plot(rx,rp/rp.max(),label='Data stacked')

param0 = [1.0, 1.4]
args = ()#(spotlist)
Result = fmin_powell(CGFOM, param0, args)
print Result
sigma = Result[0]
c=Result[1]

foocg=ConvolvedGaussian(rx,sigma,sigma, c)
plot(rx,foocg,label='Convolved Gaussian')

param0 = [1.0]
args = ()#(spotlist)
Result = fmin_powell(GFOM, param0, args)
print Result
foog=Gaussian(rx,0,sigmax=Result, sigmay=Result)
plot(rx,foog,label='Gaussian')

legend(loc='upper right')
axis([0,8,1e-4,1])
ylabel("Profile(Peak=1.0)")
xlabel("R (Pixels)")

subplot(2,1,2)
title("Error - percent full scale")
error = abs(foocg-rp/rp.max())*100.0
plot(rx,error,label='Error Convolved Gaussian')

error = abs(foog-rp/rp.max())*100.0
plot(rx,error,label='Error Gaussian')

legend(loc='upper right')
axis([0,8,1e-4,1])
ylim(0,2)
ylabel("Percent Error")
xlabel("R (Pixels)")
savefig("Drizzled_Profile_Error_05Dec17.pdf")
show()
close()


In [ ]:
def ReadCCFile(filename, nx, ny):
    file = open(filename,'r')
    lines=file.readlines()
    file.close()
    lines.remove(lines[0]) # Strip the title line     
    elec = zeros([nx,ny])
    count = 0
    for line in lines:
        items = line.split()
        i = int(items[0]) - 1
        j = int(items[1]) - 1
        elec[i,j] = int(items[2])
    return elec


def BuildSpotListLLNL(dirbase, run, numspots, nx, ny):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    for spot in range(numspots):
        filename = dirbase+'bfrun_%d/BF_%d_CC.dat'%(spot,run)
        elec = ReadCCFile(filename, nx, ny)
        xsum = 0.0
        ysum = 0.0
        datasum = 0.0
        for i in range(nx):
            for j in range(ny):
                spotlist.data[i,j,spot] = float(elec[i,j])                    
                ysum += spotlist.y[j] * spotlist.data[i,j,spot]
                xsum += spotlist.x[i] * spotlist.data[i,j,spot]
                datasum += spotlist.data[i,j,spot]
        xoff = xsum / datasum
        yoff = ysum / datasum
        spotlist.xoffset[spot] = xoff#xcoord - xint - 1.0
        spotlist.yoffset[spot] = yoff#ycoord - yint - 1.0     
    return spotlist



In [ ]:
nx = 9
ny = 9
numspots =61
run = 20
dirbase = "/Users/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole20_BF/llnl_Test11E/cc_files/"
llspotlist = BuildSpotListLLNL(dirbase, run, numspots, nx, ny)

nzoom = 10
halfstampsize = (nx + 1) / 2
llstampstack = DrizzleSpotList(llspotlist, nzoom)
[mx, my, mx2, my2, mxy] = moments(llstampstack, (45,45))
print mx, my, mx2, my2, mxy

figure(figsize=(12,6))
subplot(121)
imshow(llstampstack,cmap='rainbow',aspect='equal',origin='lower'),colorbar()

for i in range(1,2*halfstampsize):
    axvline(i*nzoom,color='w')
    axhline(i*nzoom,color='w')
axis('off')
subplot(122)
llrx,llrp=radial_profile(llstampstack,(halfstampsize*nzoom,halfstampsize*nzoom))

plot(rx,rp/rp.max(), label='Data', color='blue')
plot(llrx,llrp/llrp.max(), label='Sim', color='red')
yscale('log')

foox=np.linspace(0,8,100)
foosig=1.1
fooy=np.exp(-foox**2/(2.0*foosig**2))
plot(foox,fooy, label='Gaussian', color='green')
legend(loc='upper right')
show()
close()



In [ ]:
def LLCGFOM(params):
    [sigma, ctest] = params
    if sigma < 0.0 or ctest < 0.0: return 1.0E12
    NormFactor = CGFunc(0.0,0.0,sigma,sigma,c)
    fom = 0.0
    for i,x in enumerate(llrx):
        cg = CGFunc(x,0,sigma, sigma, ctest) / NormFactor
        fom += square(cg-llrp[i]/llrp.max())
    #print sigma, ctest, fom
    return fom

def LLGFOM(params):
    fom = 0.0
    [sigma] = params
   
    for i,x in enumerate(llrx):
        cg = Gaussian(x,0,sigmax=sigma, sigmay=sigma)
        #print x, cg, rp[i]/rp.max()
        fom += square(cg-llrp[i]/llrp.max())
    #print sigma, ctest, fom
    return fom

def LLConvolvedGaussian(rx,sigmax, sigmay,c):
    NormFactor = CGFunc(0.0,0.0,sigma,sigma,c)
    fooy = []
    for r in llrx:
        fooy.append(CGFunc(r,0.0,sigmax,sigmay,c) / NormFactor)
    return(array(fooy))




In [ ]:
figure(figsize=(12,6))
subplots_adjust(hspace=0.5)
subplot(2,1,1)
title("Spot Profile")
plot(llrx,llrp/llrp.max(),label='Data stacked')

param0 = [1.0, 1.4]
args = ()#(spotlist)
Result = fmin_powell(LLCGFOM, param0, args)
print Result
sigma = Result[0]
c=Result[1]

foocg=LLConvolvedGaussian(llrx,sigma,sigma, c)
plot(llrx,foocg,label='Convolved Gaussian')

param0 = [1.0]
args = ()#(spotlist)
Result = fmin_powell(LLGFOM, param0, args)
print Result
foog=Gaussian(llrx,0,sigmax=Result, sigmay=Result)
plot(llrx,foog,label='Gaussian')

legend(loc='upper right')
axis([0,8,1e-4,1])
ylabel("Profile(Peak=1.0)")
xlabel("R (Pixels)")

subplot(2,1,2)
title("Error - percent full scale")
error = abs(foocg-llrp/llrp.max())*100.0
plot(llrx,error,label='Error Convolved Gaussian')

error = abs(foog-llrp/llrp.max())*100.0
plot(llrx,error,label='Error Gaussian')

legend(loc='upper right')
axis([0,8,1e-4,1])
ylim(0,2)
ylabel("Percent Error")
xlabel("R (Pixels)")
savefig("Drizzled_Profile_Sim_Error_05Dec17.pdf")
show()
close()
